In [13]:
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
# RetrievalQA
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader

from langchain.callbacks import get_openai_callback
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.schema import HumanMessage, AIMessage

In [14]:
loader = PyPDFLoader('Employee-details-1.pdf')
documents = loader.load()

persist_directory = 'db'


# Now we can load the persisted database from disk, and use it as normal. 
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

 

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

qa = RetrievalQA.from_chain_type(
    llm, retriever=vectordb.as_retriever()#, chain_type_kwargs={"prompt": prompt}
)
#query = "Tell me something about each character"

prompts = [
    "What can you make of this document?",
    "Who is David Austin and what is his salary?",
    "Name an employee from China",
    "Name an employee from Europe"
]

for prompt in prompts:
    response = qa.invoke(prompt)
    print(f"Prompt: {prompt}\nResponse: {response}\n")

Prompt: What can you make of this document?
Response: {'query': 'What can you make of this document?', 'result': 'The document appears to be a database table containing information about employees. Each entry includes details such as employee ID, name, department, hire date, job title, salary, commission, manager ID, and department ID. The employees seem to be categorized into different departments such as Administration, IT, and Sales. The document provides a structured way to store and retrieve information about employees within an organization.'}

Prompt: Who is David Austin and what is his salary?
Response: {'query': 'Who is David Austin and what is his salary?', 'result': 'David Austin is an employee with an employee_id of 105. His salary is $4800.'}

Prompt: Name an employee from China
Response: {'query': 'Name an employee from China', 'result': "I don't have information on the nationality of the employees in the provided data."}

Prompt: Name an employee from Europe
Response: {'

In [15]:
def run_with_conversation_buffer(chain, query, conversation_buf):
    with get_openai_callback() as cb:
        result = chain.invoke(query)
        conversation_buf.memory.save_context({"input": query}, {"output": str(result)})
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

In [17]:
conversation_buf = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

# Example usage
queries = [
    "Who is David Austin?",
    "What is his salary?",
    "Is there someone else with the same first name?",
    "Is there someone with the same origin as him?"
]
for query in queries:
    response = run_with_conversation_buffer(qa, query, conversation_buf)
    print("Response:", response)

Spent a total of 2057 tokens
Response: {'query': 'Who is David Austin?', 'result': 'David Austin is an employee in the database provided. He has the employee ID 105 and works in the IT_PRO department. David Austin started working on June 25, 1997, and has a salary of 4800.'}
Spent a total of 930 tokens
Response: {'query': 'What is his salary?', 'result': 'The salary for Charles Johnson is 6200.'}
Spent a total of 558 tokens
Response: {'query': 'Is there someone else with the same first name?', 'result': 'Based on the provided information, there is no one else with the same first name. The first names mentioned are Christopher, Nanette, Jean, Martha, and Girard, and each of these names is unique within the given context.'}
Spent a total of 1348 tokens
Response: {'query': 'Is there someone with the same origin as him?', 'result': "I don't have enough information to determine the origin of the individuals mentioned in the context provided."}
